In [1]:
!git clone -q https://github.com/earthspecies/aves.git
!pip install -q wandb

# mount google
from google.colab import drive
drive.mount('/content/gdrive')

# 73df4adc60c07fd243c9aae5af5d6dc24878b803
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
Mounted at /content/gdrive
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
training_file = '/content/gdrive/MyDrive/thesis/train.zip'
# !cp '/content/gdrive/My Drive/thesis/data/16smalltrain.zip' '/content'
!cp '/content/gdrive/MyDrive/thesis (1)/train.zip' '/content'
# !cp '/content/gdrive/My Drive/thesis/data/16smalldev.zip' '/content'
!unzip -q '/content/train.zip' -d '/content/train'
# !unzip '/content/16smalldev.zip' -d '/content/dev'

In [3]:
!pip install datasets>=1.18.3
!pip install transformers==4.24.0
!git clone https://github.com/facebookresearch/fairseq.git
!cd fairseq
!git checkout eda70379
!cd fairseq
# !cd fairseq && echo "import fairseq.checkpoint_utils" >> fairseq/__init__.py && pip install --editable ./
!cd fairseq && echo "import fairseq.checkpoint_utils # noqa" >> fairseq/__init__.py && pip install ./

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.6 MB/s eta 0:00:00
Cloning into 'fairseq'...
remote: Enumerating objects: 34777, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 34777 (delta 0), reused 3 (delta 0), pack-reused 34769
Receiving objects: 100% (34777/34777), 25.01 MiB | 29.40 MiB/s, done.
Resolving deltas: 100% (25234/25234), done.
fatal: not a git repository (or any of the parent directories): .git
Processing /content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
!wget https://storage.googleapis.com/esp-public-files/aves/aves-base-bio.pt

--2023-09-05 19:36:21--  https://storage.googleapis.com/esp-public-files/aves/aves-base-bio.pt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.96.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1135551425 (1.1G) [application/octet-stream]
Saving to: ‘aves-base-bio.pt’

aves-base-bio.pt    100%[===================>]   1.06G  33.2MB/s    in 34s     

2023-09-05 19:36:55 (31.4 MB/s) - ‘aves-base-bio.pt’ saved [1135551425/1135551425]



In [5]:
!git clone https://github.com/mbari-org/oceansoundscape
# !pip install oceansoundscape

Cloning into 'oceansoundscape'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 53 (delta 10), reused 53 (delta 10), pack-reused 0
Receiving objects: 100% (53/53), 2.31 MiB | 20.26 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [6]:
import os
os.environ['PYTHONPATH'] += ":/content/fairseq"

import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset,Dataset,Audio
import fairseq
import torch
import torch.nn as nn
import fairseq.checkpoint_utils
import torch.optim as optim
from time import perf_counter
import wandb
import random

In [7]:
import torch.utils.checkpoint as checkpoint

class AvesClassifier(nn.Module):
    def __init__(self, model_path, num_classes, embeddings_dim=768):

        super().__init__()

        models, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([model_path])
        self.model = models[0]
        self.model.feature_extractor.requires_grad_(False)
        # use binary
        self.head = nn.Linear(in_features=embeddings_dim, out_features=num_classes)

        # nn.Sequential(
        #     nn.Linear(in_features=embeddings_dim, out_features=num_classes),
        #     nn.Softmax()
        # )

        self.loss_func = nn.CrossEntropyLoss()

    def forward(self, x, y=None):
        out = self.model.extract_features(x)[0]
        out = out.mean(dim=1)             # mean pooling, could remove
        logits = self.head(out)

        loss = None
        if y is not None:
            loss = self.loss_func(logits, y)

        return loss, logits


# Initialize an AVES classifier with 10 target classes
model = AvesClassifier(
    model_path='./aves-base-bio.pt',
    num_classes=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# if exporting need to put back on cpu
# command to watch GPU use (nvidia SMI)
# !watch -n 2 nvidia-smi
# look into opening colab terminal

AvesClassifier(
  (model): HubertModel(
    (feature_extractor): ConvFeatureExtractionModel(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): Fp32GroupNorm(512, 512, eps=1e-05, affine=True)
          (3): GELU(approximate='none')
        )
        (1-4): 4 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
        (5-6): 2 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
      )
    )
    (post_extract_proj): Linear(in_features=512, out_features=768, bias=True)
    (dropout_input): Dropout(p=0.1, inplace=False)
    (dropout_features): Dropout(p=0.1, inplace=False)
    (encoder): Tra

In [8]:
print(model)

AvesClassifier(
  (model): HubertModel(
    (feature_extractor): ConvFeatureExtractionModel(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): Fp32GroupNorm(512, 512, eps=1e-05, affine=True)
          (3): GELU(approximate='none')
        )
        (1-4): 4 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
        (5-6): 2 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
      )
    )
    (post_extract_proj): Linear(in_features=512, out_features=768, bias=True)
    (dropout_input): Dropout(p=0.1, inplace=False)
    (dropout_features): Dropout(p=0.1, inplace=False)
    (encoder): Tra

In [9]:
# count = 0
# for child in model.children():
#   if count==0:
#     # for child in child.children():
#     #   if count==0:
#     #     # print(child)
#     #     continue
#     #   count += 1
#     # print(count)
#     for param in child.parameters():
#       if count<4:
#         continue
#       count += 1
#       param.requires_grad = True

#     else:
#       for param in child.parameters():
#         param.requires_grad = True
#   count += 1

In [10]:
# torch.cuda.is_available()
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [11]:
# train_dataset = load_dataset("audiofolder", data_dir='/content/train/content/train', split='train[:10%]+train[-10%:]') # use 10%
# dev_dataset = load_dataset("audiofolder", data_dir='/content/train/content/train', split='train[10%:20%]+train[-20%:-10%]')

# train_dataloader= torch.utils.data.DataLoader(dataset=train_dataset, batch_size=16,shuffle=True, num_workers=2)
# test_dataloader= torch.utils.data.DataLoader(dataset=dev_dataset, batch_size=16,shuffle=True, num_workers=2)


In [12]:
def loadData(batch, samples):
  train_dataset = load_dataset("audiofolder", data_dir='/content/train/content', split=f'train[:{samples}]+train[-{samples}:]') # use 10%
  dev_dataset = load_dataset("audiofolder", data_dir='/content/train/content', split=f'train[{samples}:{2*samples}]+train[-{2*samples}:-{samples}]')

  train_dataloader= torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch,shuffle=True, num_workers=2)
  test_dataloader= torch.utils.data.DataLoader(dataset=dev_dataset, batch_size=batch,shuffle=True, num_workers=2)

  return train_dataloader, test_dataloader

def save_model(model):
  torch.save(model.state_dict(), 'blued.pt')
  !cp '/content/blued.pt' '/content/gdrive/MyDrive/thesis/data'

def train(learning_rate, num_epochs, train_dataloader, test_dataloader, model, num_accumulation_steps):
  criterion= nn.CrossEntropyLoss()
  optimizer=optim.Adam(model.parameters(), lr=learning_rate)

  t1_start = perf_counter()
  losses = np.zeros(num_epochs)

  for epoch in range(num_epochs):
      running_loss=0.0
      for i, data in enumerate(train_dataloader,0):
          audioarrs = data['audio']['array'].to(device).to(torch.float32)
          labels=data['label'].to(device)

          # zero the parameter gradients
          # optimizer.zero_grad()

          # forward
          loss,logits= model(audioarrs)
          #calculate the loss between the target and the actuals
          loss= criterion(logits, labels)
          #calculate loss
          loss = loss / num_accumulation_steps
          loss.backward()

          if ((i + 1) % num_accumulation_steps == 0) or (i + 1 == len(train_dataloader)):
            optimizer.step()
            # Reset gradient tensors
            optimizer.zero_grad()

      print('epoch %d - Loss %.3f' % (epoch +1, loss.item()))

      losses[epoch] = loss.item()
      accuracy = test_model(model, test_dataloader)
      wandb.log({"loss": loss.item(), "test_loss": accuracy})

  t1_end = perf_counter()
  print("Time for training using PyTorch %f" %(t1_end-t1_start))
  return losses

In [13]:
def test_model(model, test_dataloader):
  #evaluate on test dataset
  correct=0
  total=0
  t1_start=perf_counter()
  with torch.no_grad():
    count = 0
    for data in test_dataloader:
        count += 1
        if count%10 == 0:
          break
          print('.',end='',flush=True)
        audioarrs = data['audio']['array'].to(device).to(torch.float32)
        labels=data['label'].to(device)
        loss,logits= model(audioarrs)
        _,pred= torch.max(logits,1)
        total+=labels.size(0)
        correct += (pred == labels).sum().item()

  t1_end=perf_counter()
  accuracy = 100 * correct / total
  print("Eval accuracy using PyTorch is %.2f and execution time %.2f seconds" %((100 * (correct / total)), (t1_end-t1_start)))
  return accuracy

In [14]:
# start a new wandb run to track this script
def initdb(lr, epochs, numsamples, batchsize):
  wandb.init(
      # set the wandb project where this run will be logged
      project="blued",

      # track hyperparameters and run metadata
      config={
      "learning_rate": lr,
      "architecture": "transformer",
      "dataset": "smallbluedlowpass",
      "epochs": epochs,
      "num_samples": numsamples,
      "batch_size": batchsize,
      }
  )

# wandb.log({"acc": acc, "loss": loss})
# wandb.finish()
# make sure it's logging correctly (make sure data/loss is plottable)
# spectrogramify the new audio clips

In [19]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

def sweep():
  lr = 0.01
  epochs = 20
  batch = 2
  samples = 500
  grad_steps = 8
  train_dataloader, test_dataloader = loadData(samples, batch)
  initdb(lr, epochs, samples, batch)
  train(lr, epochs, train_dataloader, test_dataloader, model, grad_steps)
  test_model(model, test_dataloader)
  # save_model(model)
  wandb.finish()

In [20]:
sweep()

Resolving data files:   0%|          | 0/1845 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1845 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [24]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 4            |        cudaMalloc retries: 4         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  38086 MiB |  38123 MiB |  85555 MiB |  47468 MiB |
|       from large pool |  38081 MiB |  38118 MiB |  85511 MiB |  47429 MiB |
|       from small pool |      5 MiB |      6 MiB |     44 MiB |     39 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  38086 MiB |  38123 MiB |  85555 MiB |  47468 MiB |
|       from large pool |  38081 MiB |  38118 MiB |  85511 MiB |

In [17]:
torch.cuda.empty_cache()
# model.to('cuda')

In [ ]:
# sample = torch.from_numpy(np.array([train_dataset['train'][x]['audio']['array'] for x in range(2)]))
# sample = torch.from_numpy(train_dataset['train'][0]['audio']['array']).unsqueeze(0)
# print(sample)
# y = torch.tensor([0])
# loss,logits = model(sample,y)

# torch.rand((16_000)).unsqueeze(0)

In [ ]:
# !cp '/content/gdrive/MyDrive/thesis/data/blued.pt' '/content/'
# model.load_state_dict(torch.load('/content/blued.pt'))
# model.eval()

### notebook updates

make everything into functions
organize data better
disable gradient calculation in evaluation?
other optimizations for cuda stuff
generate spectrograms of new data
build the validation set + full training set
build smaller training/val set to avoid unzipping full file

In [ ]:
#evaluate on test dataset
correct=0
total=0
t1_start=perf_counter()
with torch.no_grad():

    '''Disabling gradient calculation is useful for inference, when you are sure that you will not call Tensor.backward().
    It will reduce memory consumption for computations that would otherwise have requires_grad=True.'''
count = 0
for  data in test_dataloader:
    count += 1
    if count%10 == 0:
      break
      print('.',end='',flush=True)
    audioarrs = data['audio']['array'] # list(map(lambda x: x['array'], data['audio']))
    labels=data['label']
    loss,logits= model(audioarrs)
    _,pred= torch.max(logits,1)
    total+=labels.size(0)
    correct += (pred == labels).sum().item()

t1_end=perf_counter()
print("Eval accuracy using PyTorch is %.2f and execution time %.2f seconds" %((100 * (correct / total)), (t1_end-t1_start)))

In [ ]:
# function ClickConnect(){
# console.log("Working");
# document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# setInterval(ClickConnect,60000)

# print(audio_dataset['train']['audio'][0])
# print(audio_dataset['train']['label'][0])